In [12]:
# Imports

import os
import cv2
import random
import pandas as pd
import glob
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Image, display
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

%matplotlib inline

In [13]:
# Constants

image_path = "images"
colors = ["red", "green", "yellow"]
image_size = 32
test_size = 100

In [14]:
# Data loading (and reshaping)

dataset = []

for color in colors:
    for im in glob.glob(os.path.join(image_path, color, "*")):
        image = cv2.imread(im)
        if not image is None:
            image = cv2.resize(image, (image_size, image_size))
            assert(image.shape == (32, 32, 3))
            dataset.append((image, color, im))

random.shuffle(dataset)

In [15]:
#Useful functions

def print_image(dataset, i):
    image_path = dataset[i][2]
    ex_img = Image(filename=image_path)
    display(ex_img)

def brightness_value(image):
    value = 0
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    new_hsv = hsv[3:image_size-3,12:image_size-12]
    maximum_brightness = np.argmax(get_max_from_hsv(new_hsv))
    return 3 + maximum_brightness

def sort_train_test(dataset, test_size):
    
    # Initialisation of arrays
    
    dataframe_values_train = np.empty(len(dataset) - test_size, dtype=int)
    dataframe_values_test = np.empty(test_size, dtype=int)
    image_colors_train = np.empty(len(dataset) - test_size, dtype=object)
    image_colors_test = np.empty(test_size, dtype=object)
    
    # Creating X_train and y_train
    
    for i in range(test_size, test_size + np.size(dataframe_values_train)):
        image = dataset[i][0]
        color = dataset[i][1]
        dataframe_values_train[test_size - i] = brightness_value(image)
        image_colors_train[test_size - i] = color
    
    X_train = pd.DataFrame(dataframe_values_train, columns=['highest_brightness_height'])
    y_train = pd.Series(image_colors_train)
    
    # Creating X_test and y_test
    
    for i in range(np.size(dataframe_values_test)):
        image = dataset[i][0]
        color = dataset[i][1]
        dataframe_values_test[i] = brightness_value(image)
        image_colors_test[i] = color
    
    X_test = pd.DataFrame(dataframe_values_test, columns=['highest_brightness_height'])
    y_test = pd.Series(image_colors_test)
    
    return X_train, y_train, X_test, y_test

def get_max_from_hsv(hsv_image):
    intermediary_max = []
    for i in range(len(hsv_image)):
        intermediary_max.append(np.max(hsv_image[i]))
    return intermediary_max

def color_to_int(color):
    if color == 'red': return 1
    if color == 'green': return 2
    if color == 'yellow': return 3

In [16]:
X_train, y_train, X_test, y_test = sort_train_test(dataset, test_size)

classifier = KNeighborsClassifier().fit(X_train, y_train)
print(X_test)
print(y_test)

    highest_brightness_height
0                           6
1                           6
2                          27
3                           8
4                           7
..                        ...
95                          9
96                          9
97                          6
98                          8
99                         23

[100 rows x 1 columns]
0       red
1       red
2     green
3       red
4       red
      ...  
95      red
96      red
97      red
98      red
99    green
Length: 100, dtype: object


In [17]:
y_pred = classifier.predict(X_test)
nb_mistakes = 0
for i in range(test_size):
    if y_pred[i] != y_test[i]:
        nb_mistakes +=1
        print('At index ' + str(i) + ' color ' + y_pred[i] + ' was predicted but color ' + y_test[i] + ' was correct.')
print('Number of mistakes: ' + str(nb_mistakes) + ' out of ' + str(test_size))
score = 100 - (nb_mistakes / test_size) * 100
print('Effectiveness : ' + str(score) + '% correct answers')

At index 36 color red was predicted but color green was correct.
At index 91 color green was predicted but color red was correct.
Number of mistakes: 2 out of 100
Effectiveness : 98.0% correct answers


C:\Users\thoma\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


CNN-method

In [18]:
train_data = keras.preprocessing.image_dataset_from_directory(
    image_path,
    batch_size=32,
    image_size=(32, 32),
    validation_split=0.2,
    subset="training",
    seed=123,
)

Found 3016 files belonging to 6 classes.
Using 2413 files for training.


In [19]:
normalization_layer = layers.experimental.preprocessing.Rescaling(1./255)
normalized_train_data = train_data.map(lambda x, y: (normalization_layer(x), y))

In [20]:
model = keras.Sequential([
    layers.Conv2D(32, 3, activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(128, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(3, activation='softmax')
])

In [21]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [22]:
path_to_test_folder = "images/test"
list_of_results = []
for filename in os.listdir(path_to_test_folder):
    new_image = keras.preprocessing.image.load_img(path_to_test_folder + "/" + filename, target_size=(32, 32))
    new_image_array = keras.preprocessing.image.img_to_array(new_image)
    new_image_array = tf.expand_dims(new_image_array, 0) / 255.0
    prediction = model.predict(new_image_array)
    colors = ["red", "yellow", "green", "multicolor"]
    predicted_color = colors[prediction.argmax()]
    list_of_results.append([filename, predicted_color])

#print("The predicted color is : ", predicted_color)
print(list_of_results)

1/1 [==============================] - 0s 16ms/step
[['00febbe1-a9ae-4b5f-b682-8ebfdae485a3.jpg', 'yellow'], ['01ae3c3d-21c8-4711-853a-ba6fda9553bf.jpg', 'yellow'], ['01d76b8c-dc66-47b6-83d4-b00826dfec18.jpg', 'green'], ['0a251f2f-babd-4257-87be-2940bd4bacc8.jpg', 'yellow'], ['0ab8c5a1-a750-4137-ad0a-13e5da55bd09.jpg', 'yellow'], ['0acf7bce-03d1-46af-aac2-bbca22e1c1cb.jpg', 'yellow'], ['0ae63733-c453-42b2-89cf-e4a7877feda6.jpg', 'yellow'], ['0aec12bd-ce95-4400-937b-ece1fd7c3011.jpg', 'yellow'], ['0b2b43f5-cad6-41a3-80ab-baf02e9f7ff1.jpg', 'yellow'], ['0b3606b7-bf9e-49d8-8de8-801bb8374b2d.jpg', 'yellow'], ['0b534a19-781b-4883-8005-4dce2478e772.jpg', 'yellow'], ['0b602dec-60ad-4601-ba50-30850e9b6895.jpg', 'yellow'], ['0ba83f83-a49c-4319-8ee2-547082ef7503.jpg', 'yellow'], ['0bbaa6ca-9f78-4cad-8c22-b95648fb90be.jpg', 'yellow'], ['0bc2c699-227a-4ac8-a7f4-8944c210d885.jpg', 'green'], ['0bc8252c-d3f6-4ef4-b4aa-d9f46f28f137.jpg', 'yellow'], ['0bda4389-2f4a-45fe-8597-c6918a6d15b5.jpg', 'green']